# Ερώτηση 4

In [1]:
import pandas as pd
import numpy as np
import json
import time 

dataset_path = '../yelp-dataset/' # Change this path to match your local dataset folder path

business_filename = 'yelp_academic_dataset_business.json'
users_filename = 'yelp_academic_dataset_user.json'

In [2]:
list_business = []
with open(dataset_path + business_filename, 'r', encoding="utf8") as buisness_file:
    for line in buisness_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 10:
            list_business.append([json_dict['business_id'], json_dict['name'], json_dict['review_count']]) 

In [3]:
df_buisness = pd.DataFrame(list_business, columns=['business_id', 'name', 'reviews_count'])
display(df_buisness)

,business_id,name,reviews_count
0,f9NumwFMBDn751xgFiRbNA,The Range At Lake Norman,36
1,51M2Kk903DFYI6gnB5I6SQ,USE MY GUY SERVICES LLC,26
2,cKyLV5oWZJ2NudWgqs8VZw,Oasis Auto Center - Gilbert,38
3,oiAlXZPIFm2nBCt0DHLu_Q,Green World Cleaners,81
4,ScYkbYNkDgCneBrD9vqhCQ,Junction Tire & Auto Service,18
...,...,...,...
103798,BAVuLTDmpSzDCk37A5HjtQ,Bruegger's Bagels,13
103799,hskVqZCPqy-omm9CHi44xQ,Domino's,16
103800,JjcJVqhZXhP4tvOhg3fnag,Water Heater Pros,217
103801,PotiAhtDMYMNsJCzV0oLAw,Walmart,13


In [4]:
graph_G = {} # dictionary whose keys are the nodes of the graph. 
             # For each key, the corresponding value is a list 
             # containing the nodes that are connected by a direct 
             # arc from this node.

In [25]:
list_users = []
with open(dataset_path + users_filename, 'r', encoding="utf8") as users_file:
    for line in users_file:
        json_dict = json.loads(line)
        if json_dict['review_count'] >= 10 and json_dict['friends'] != 'None': 
            list_users.append([json_dict['user_id'], json_dict['name'], json_dict['review_count'], json_dict['friends'].replace(',', '').split()])
            graph_G[json_dict['user_id']] = json_dict['friends'].replace(',', '').split() # converts long string of id's to a list.

In [26]:
df_users = pd.DataFrame(list_users, columns=['business_id', 'name', 'review_count', 'friends'])
display(df_users)

,business_id,name,review_count,friends
0,ntlvfPzc8eglqvk92iDIAw,Rafael,553,"[oeMvJh94PiGQnx_6GlndPQ, wm1z1PaJKvHgSDRKfwhfD..."
1,FOBRPlBHa3WPHFB5qYDlVg,Michelle,564,"[ly7EnE8leJmyqyePVYFlug, pRlR63iDytsnnniPb3AOu..."
2,zZUnPeh2hEp0WydbAZEOOg,Martin,60,"[Uwlk0txjQBPw_JhHsQnyeg, Ybxr1tSCkv3lYA0I1qmnP..."
3,QaELAmRcDc5TfJEylaaP8g,John,206,"[iog3Nyg1i4jeumiTVG_BSA, M92xWY2Vr9w0xoH8bPplf..."
4,xvu8G900tezTzbbfqmTKvA,Anne,485,"[3W3ZMSthojCUirKEqAwGNw, eTIbuu23j9tOgmIa9POyL..."
...,...,...,...,...
532942,oz0eh8MPenJvloapPJd6Iw,Frank,21,[9djH4FrXXbuv0ogIqNsWgA]
532943,XXstm7H54zEy0eqSwV6xOg,Charlene,14,[cqTGdXWgS1_edskd-cJPCA]
532944,DQQ-AngIae2DrpapnFaP3g,Cindy,12,[ezfEXtW5-pVFuwV0bnpBaw]
532945,3nLD8UrOXRQYb1GNaSxoYA,Jessica,32,[tDYysNkQGmEm1iJONRfruw]


In [93]:
# Testing - connected components
graph = {
    'A' : ['B','C'],
    'B' : ['D', 'E'],
    'C' : ['F'],
    'D' : [],
    'E' : ['F'],
    'F' : [],
    'E1' : ['F1'],
    'F1' : []
}

def dfs(visited, graph, node):
    if node not in visited and node in graph.keys(): # skip missing nodes we trimmed in previous step
        #print (node)
        visited.add(node)
        for neighbour in graph[node]:
            dfs(visited, graph, neighbour)

def iterative_dfs(visited, graph, node):
    stack = []
    stack.append(node) 

    while (len(stack)): 
        node = stack[-1] 
        stack.pop()

        if node not in visited and node in graph.keys(): 
            #print(node, end=' ')
            visited.add(node)

        for node in graph[node]: 
            if node not in visited and node in graph.keys(): 
                stack.append(node) 

def connected_components(graph):
    vertices_counter = 0
    components_counter = 0
    visited = set() # Set to keep track of visited nodes.
    for vertex in graph.keys():
        if vertex not in visited:
            iterative_dfs(visited, graph, vertex)
            components_counter += 1
            vertices_counter = len(visited) - vertices_counter 
            print("CC {}, size {}\n".format(components_counter, vertices_counter))

In [94]:
connected_components(graph)

CC 1, size 6

CC 2, size 2

